<a href="https://colab.research.google.com/github/shinobu357/TugasMLRaisya/blob/main/Week%2011/Week_11_Raisya_Athaya_Kamilah_101032380253_HeartDisease_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive

# Hubungkan ke Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [6]:
# Membaca file dataset yang diupload
df = pd.read_csv('/content/drive/MyDrive/Week 11/heart.csv')  # Gantilah dengan nama file yang benar

# Menampilkan informasi dataset
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


# Preprocessing Data

In [7]:
# Pisahkan fitur dan target
X = df.drop(columns=['target']).values
y = df['target'].values

# Pembagian data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Konversi data ke tensor PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # reshape untuk output biner
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [8]:
# MLP dengan activation yang berbeda
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLP, self).__init__()
        layers = []
        prev_size = input_size

        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            if activation_fn == 'ReLU':
                layers.append(nn.ReLU())
            elif activation_fn == 'Sigmoid':
                layers.append(nn.Sigmoid())
            elif activation_fn == 'Tanh':
                layers.append(nn.Tanh())
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, 1))  # Output layer
        layers.append(nn.Sigmoid())  # Untuk klasifikasi biner
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [9]:
def train_model(model, criterion, optimizer, epochs, batch_size):
    best_accuracy = 0
    accuracies = []

    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(X_train.size(0))  # Mengacak data untuk batch

        epoch_loss = 0
        for i in range(0, X_train.size(0), batch_size):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            # Menambahkan loss setiap batch untuk menghitung rata-rata loss per epoch
            epoch_loss += loss.item()

        # Rata-rata loss untuk satu epoch
        epoch_loss /= len(X_train) / batch_size

        # Menghitung akurasi setelah setiap epoch
        model.eval()
        with torch.no_grad():
            outputs = model(X_test)
            predicted = (outputs > 0.5).float()  # Threshold 0.5 untuk klasifikasi biner
            accuracy = (predicted == y_test).float().mean().item()
            accuracies.append(accuracy)

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_epoch = epoch
                best_batch_size = batch_size

        # Print progress setiap epoch
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

    return best_accuracy, best_epoch, best_batch_size, accuracies


# MLP Model dengan Activation Function (ReLU)

In [ ]:
# Hyperparameters untuk eksperimen
hidden_layer_sizes = [4,8,16, 32, 64]  # Jumlah neuron hidden layers
epochs_list = [1,10, 25, 50,100,250]  # Jumlah epoch
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]  # Learning rates
batch_sizes = [16,32, 64, 128,256,512]  # Batch sizes

# Hasil eksperimen ReLU
results_relu = []

# Menggunakan kombinasi hidden layers untuk eksperimen
for hidden_size in hidden_layer_sizes:
    for epoch in epochs_list:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Training with ReLU activation, {hidden_size} hidden units, {epoch} epochs, {lr} learning rate, {batch_size} batch size")

                # Inisialisasi model
                model = MLP(input_size=X_train.shape[1], hidden_layers=[hidden_size], activation_fn='ReLU')
                criterion = nn.BCELoss()  # Binary Cross-Entropy loss
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training model
                best_accuracy, best_epoch, best_batch_size, accuracies = train_model(
                    model, criterion, optimizer, epoch, batch_size)

                # Simpan hasil untuk analisis
                results_relu.append({
                    "hidden_size": hidden_size,
                    "epochs": epoch,
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "accuracy": best_accuracy
                })

# Tampilkan Top 5 Hasil Terbaik ReLU berdasarkan akurasi
df_relu = pd.DataFrame(results_relu)
top_5_relu = df_relu.nlargest(5, 'accuracy')
print("\nTop 5 Best Results for ReLU Activation:")
print(top_5_relu)

# Tampilkan hasil eksperimen berdasarkan jumlah hidden layer dan neuron
print("\nGrouped Results Based on Hidden Layer Size and Neuron Count:")
grouped_results = df_relu.groupby(['hidden_size', 'epochs', 'learning_rate', 'batch_size'])['accuracy'].max()
print(grouped_results)



Streaming output truncated to the last 5000 lines.
Epoch 43/250, Loss: 0.0765, Accuracy: 0.9512
Epoch 44/250, Loss: 0.0778, Accuracy: 0.9463
Epoch 45/250, Loss: 0.0665, Accuracy: 0.9463
Epoch 46/250, Loss: 0.0722, Accuracy: 0.9561
Epoch 47/250, Loss: 0.0582, Accuracy: 0.9561
Epoch 48/250, Loss: 0.0556, Accuracy: 0.9561
Epoch 49/250, Loss: 0.0608, Accuracy: 0.9463
Epoch 50/250, Loss: 0.0503, Accuracy: 0.9805
Epoch 51/250, Loss: 0.0493, Accuracy: 0.9805
Epoch 52/250, Loss: 0.0566, Accuracy: 0.9805
Epoch 53/250, Loss: 0.0492, Accuracy: 0.9707
Epoch 54/250, Loss: 0.0461, Accuracy: 0.9561
Epoch 55/250, Loss: 0.0465, Accuracy: 0.9805
Epoch 56/250, Loss: 0.0455, Accuracy: 0.9805
Epoch 57/250, Loss: 0.0402, Accuracy: 0.9659
Epoch 58/250, Loss: 0.0386, Accuracy: 0.9561
Epoch 59/250, Loss: 0.0471, Accuracy: 0.9805
Epoch 60/250, Loss: 0.0367, Accuracy: 0.9854
Epoch 61/250, Loss: 0.0373, Accuracy: 0.9854
Epoch 62/250, Loss: 0.0387, Accuracy: 0.9805
Epoch 63/250, Loss: 0.0328, Accuracy: 0.9805
Epoc

# Sigmoid

In [ ]:
# Hasil eksperimen Sigmoid
results_sigmoid = []

# Menggunakan kombinasi hidden layers untuk eksperimen
for hidden_size in hidden_layer_sizes:
    for epoch in epochs_list:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Training with Sigmoid activation, {hidden_size} hidden units, {epoch} epochs, {lr} learning rate, {batch_size} batch size")

                # Inisialisasi model
                model = MLP(input_size=X_train.shape[1], hidden_layers=[hidden_size], activation_fn='Sigmoid')
                criterion = nn.BCELoss()  # Binary Cross-Entropy loss
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training model
                best_accuracy, best_epoch, best_batch_size, accuracies = train_model(
                    model, criterion, optimizer, epoch, batch_size)

                # Simpan hasil untuk analisis
                results_sigmoid.append({
                    "hidden_size": hidden_size,
                    "epochs": epoch,
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "accuracy": best_accuracy
                })

# Tampilkan Top 5 Hasil Terbaik Sigmoid berdasarkan akurasi
df_sigmoid = pd.DataFrame(results_sigmoid)
top_5_sigmoid = df_sigmoid.nlargest(5, 'accuracy')
print("\nTop 5 Best Results for Sigmoid Activation:")
print(top_5_sigmoid)

# Tampilkan hasil eksperimen berdasarkan jumlah hidden layer dan neuron
print("\nGrouped Results Based on Hidden Layer Size and Neuron Count for Sigmoid:")
grouped_results_sigmoid = df_sigmoid.groupby(['hidden_size', 'epochs', 'learning_rate', 'batch_size'])['accuracy'].max()
print(grouped_results_sigmoid)


Streaming output truncated to the last 5000 lines.
Epoch 43/250, Loss: 0.3918, Accuracy: 0.8049
Epoch 44/250, Loss: 0.3592, Accuracy: 0.8195
Epoch 45/250, Loss: 0.3736, Accuracy: 0.8195
Epoch 46/250, Loss: 0.4064, Accuracy: 0.8195
Epoch 47/250, Loss: 0.3512, Accuracy: 0.8146
Epoch 48/250, Loss: 0.3656, Accuracy: 0.8146
Epoch 49/250, Loss: 0.3359, Accuracy: 0.8049
Epoch 50/250, Loss: 0.3837, Accuracy: 0.8098
Epoch 51/250, Loss: 0.3619, Accuracy: 0.8195
Epoch 52/250, Loss: 0.3429, Accuracy: 0.8244
Epoch 53/250, Loss: 0.3352, Accuracy: 0.8195
Epoch 54/250, Loss: 0.3598, Accuracy: 0.8195
Epoch 55/250, Loss: 0.3836, Accuracy: 0.8098
Epoch 56/250, Loss: 0.3199, Accuracy: 0.8146
Epoch 57/250, Loss: 0.3882, Accuracy: 0.8195
Epoch 58/250, Loss: 0.3535, Accuracy: 0.8195
Epoch 59/250, Loss: 0.3391, Accuracy: 0.8195
Epoch 60/250, Loss: 0.3526, Accuracy: 0.8341
Epoch 61/250, Loss: 0.3355, Accuracy: 0.8341
Epoch 62/250, Loss: 0.3741, Accuracy: 0.8341
Epoch 63/250, Loss: 0.3458, Accuracy: 0.8439
Epoc

# Tanh

In [ ]:
# Hasil eksperimen Tanh
results_tanh = []

# Menggunakan kombinasi hidden layers untuk eksperimen
for hidden_size in hidden_layer_sizes:
    for epoch in epochs_list:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Training with Tanh activation, {hidden_size} hidden units, {epoch} epochs, {lr} learning rate, {batch_size} batch size")

                # Inisialisasi model
                model = MLP(input_size=X_train.shape[1], hidden_layers=[hidden_size], activation_fn='Tanh')
                criterion = nn.BCELoss()  # Binary Cross-Entropy loss
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training model
                best_accuracy, best_epoch, best_batch_size, accuracies = train_model(
                    model, criterion, optimizer, epoch, batch_size)

                # Simpan hasil untuk analisis
                results_tanh.append({
                    "hidden_size": hidden_size,
                    "epochs": epoch,
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "accuracy": best_accuracy
                })

# Tampilkan Top 5 Hasil Terbaik Tanh berdasarkan akurasi
df_tanh = pd.DataFrame(results_tanh)
top_5_tanh = df_tanh.nlargest(5, 'accuracy')
print("\nTop 5 Best Results for Tanh Activation:")
print(top_5_tanh)

# Tampilkan hasil eksperimen berdasarkan jumlah hidden layer dan neuron
print("\nGrouped Results Based on Hidden Layer Size and Neuron Count for Tanh:")
grouped_results_tanh = df_tanh.groupby(['hidden_size', 'epochs', 'learning_rate', 'batch_size'])['accuracy'].max()
print(grouped_results_tanh)


Streaming output truncated to the last 5000 lines.
Epoch 43/250, Loss: 0.1479, Accuracy: 0.8976
Epoch 44/250, Loss: 0.1203, Accuracy: 0.9073
Epoch 45/250, Loss: 0.1369, Accuracy: 0.9171
Epoch 46/250, Loss: 0.1305, Accuracy: 0.9171
Epoch 47/250, Loss: 0.1087, Accuracy: 0.9268
Epoch 48/250, Loss: 0.1026, Accuracy: 0.9463
Epoch 49/250, Loss: 0.1012, Accuracy: 0.9463
Epoch 50/250, Loss: 0.0900, Accuracy: 0.9463
Epoch 51/250, Loss: 0.0800, Accuracy: 0.9512
Epoch 52/250, Loss: 0.0846, Accuracy: 0.9512
Epoch 53/250, Loss: 0.0738, Accuracy: 0.9512
Epoch 54/250, Loss: 0.0715, Accuracy: 0.9512
Epoch 55/250, Loss: 0.0748, Accuracy: 0.9561
Epoch 56/250, Loss: 0.0647, Accuracy: 0.9561
Epoch 57/250, Loss: 0.0631, Accuracy: 0.9659
Epoch 58/250, Loss: 0.0612, Accuracy: 0.9659
Epoch 59/250, Loss: 0.0603, Accuracy: 0.9659
Epoch 60/250, Loss: 0.0529, Accuracy: 0.9659
Epoch 61/250, Loss: 0.0541, Accuracy: 0.9659
Epoch 62/250, Loss: 0.0490, Accuracy: 0.9659
Epoch 63/250, Loss: 0.0449, Accuracy: 0.9659
Epoc

# Softmax

In [10]:
hidden_layer_sizes = [4,8,16, 32, 64]  # Jumlah neuron hidden layers
epochs_list = [1,10, 25, 50,100,250]  # Jumlah epoch
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]  # Learning rates
batch_sizes = [16,32, 64, 128,256,512]  #

# Hasil eksperimen Softmax
results_softmax = []

# Menggunakan kombinasi hidden layers untuk eksperimen
for hidden_size in hidden_layer_sizes:
    for epoch in epochs_list:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Training with Softmax activation, {hidden_size} hidden units, {epoch} epochs, {lr} learning rate, {batch_size} batch size")

                # Inisialisasi model
                model = MLP(input_size=X_train.shape[1], hidden_layers=[hidden_size], activation_fn='Softmax')
                criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss digunakan untuk softmax
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training model
                best_accuracy, best_epoch, best_batch_size, accuracies = train_model(
                    model, criterion, optimizer, epoch, batch_size)

                # Simpan hasil untuk analisis
                results_softmax.append({
                    "hidden_size": hidden_size,
                    "epochs": epoch,
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "accuracy": best_accuracy
                })

# Tampilkan Top 5 Hasil Terbaik Softmax berdasarkan akurasi
df_softmax = pd.DataFrame(results_softmax)
top_5_softmax = df_softmax.nlargest(5, 'accuracy')
print("\nTop 5 Best Results for Softmax Activation:")
print(top_5_softmax)

# Tampilkan hasil eksperimen berdasarkan jumlah hidden layer dan neuron
print("\nGrouped Results Based on Hidden Layer Size and Neuron Count for Softmax:")
grouped_results_softmax = df_softmax.groupby(['hidden_size', 'epochs', 'learning_rate', 'batch_size'])['accuracy'].max()
print(grouped_results_softmax)


Streaming output truncated to the last 5000 lines.
Epoch 43/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 44/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 45/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 46/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 47/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 48/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 49/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 50/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 51/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 52/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 53/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 54/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 55/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 56/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 57/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 58/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 59/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 60/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 61/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 62/250, Loss: 0.0000, Accuracy: 0.4049
Epoch 63/250, Loss: 0.0000, Accuracy: 0.4049
Epoc

# Linear

In [11]:
hidden_layer_sizes = [4,8,16, 32, 64]  # Jumlah neuron hidden layers
epochs_list = [1,10, 25, 50,100,250]  # Jumlah epoch
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]  # Learning rates
batch_sizes = [16,32, 64, 128,256,512]  #
# Hasil eksperimen Linear
results_linear = []

# Menggunakan kombinasi hidden layers untuk eksperimen
for hidden_size in hidden_layer_sizes:
    for epoch in epochs_list:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Training with Linear activation, {hidden_size} hidden units, {epoch} epochs, {lr} learning rate, {batch_size} batch size")

                # Inisialisasi model
                model = MLP(input_size=X_train.shape[1], hidden_layers=[hidden_size], activation_fn='Linear')
                criterion = nn.BCELoss()  # Binary Cross-Entropy loss
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training model
                best_accuracy, best_epoch, best_batch_size, accuracies = train_model(
                    model, criterion, optimizer, epoch, batch_size)

                # Simpan hasil untuk analisis
                results_linear.append({
                    "hidden_size": hidden_size,
                    "epochs": epoch,
                    "learning_rate": lr,
                    "batch_size": batch_size,
                    "accuracy": best_accuracy
                })

# Tampilkan Top 5 Hasil Terbaik Linear berdasarkan akurasi
df_linear = pd.DataFrame(results_linear)
top_5_linear = df_linear.nlargest(5, 'accuracy')
print("\nTop 5 Best Results for Linear Activation:")
print(top_5_linear)

# Tampilkan hasil eksperimen berdasarkan jumlah hidden layer dan neuron
print("\nGrouped Results Based on Hidden Layer Size and Neuron Count for Linear:")
grouped_results_linear = df_linear.groupby(['hidden_size', 'epochs', 'learning_rate', 'batch_size'])['accuracy'].max()
print(grouped_results_linear)


Streaming output truncated to the last 5000 lines.
Epoch 43/250, Loss: 0.4149, Accuracy: 0.8000
Epoch 44/250, Loss: 0.4137, Accuracy: 0.8049
Epoch 45/250, Loss: 0.4046, Accuracy: 0.8146
Epoch 46/250, Loss: 0.4172, Accuracy: 0.8098
Epoch 47/250, Loss: 0.4175, Accuracy: 0.8000
Epoch 48/250, Loss: 0.3912, Accuracy: 0.7805
Epoch 49/250, Loss: 0.3967, Accuracy: 0.7951
Epoch 50/250, Loss: 0.4200, Accuracy: 0.7805
Epoch 51/250, Loss: 0.4161, Accuracy: 0.7805
Epoch 52/250, Loss: 0.4165, Accuracy: 0.7951
Epoch 53/250, Loss: 0.4138, Accuracy: 0.8000
Epoch 54/250, Loss: 0.4390, Accuracy: 0.7902
Epoch 55/250, Loss: 0.3901, Accuracy: 0.7951
Epoch 56/250, Loss: 0.4272, Accuracy: 0.7902
Epoch 57/250, Loss: 0.4311, Accuracy: 0.7902
Epoch 58/250, Loss: 0.4173, Accuracy: 0.8049
Epoch 59/250, Loss: 0.4113, Accuracy: 0.8000
Epoch 60/250, Loss: 0.4514, Accuracy: 0.8000
Epoch 61/250, Loss: 0.4155, Accuracy: 0.7854
Epoch 62/250, Loss: 0.3842, Accuracy: 0.7902
Epoch 63/250, Loss: 0.4114, Accuracy: 0.7902
Epoc